In [1]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

In [2]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

In [4]:
train = utils.image_dataset_from_directory("datasets/data/split/VTID2/train/", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("datasets/data/split/VTID2/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("datasets/data/split/VTID2/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 3048 files belonging to 5 classes.
Found 652 files belonging to 5 classes.
Found 656 files belonging to 5 classes.


In [5]:
inputs = keras.Input(shape=(100, 100, 3))
x = layers.Conv2D(32, 3, activation='relu')(inputs)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(5, activation='softmax')(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 128)       73856 

In [6]:
history = model.fit(
  train.cache(),
  epochs=40,
  validation_data=val.cache()
)

Epoch 1/40


2022-08-01 10:37:59.047672: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-01 10:37:59.047757: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


96/96 [==============================] - ETA: 0s - loss: 2.6805 - accuracy: 0.2940

2022-08-01 10:38:03.820058: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


96/96 [==============================] - 6s 52ms/step - loss: 2.6805 - accuracy: 0.2940 - val_loss: 1.9913 - val_accuracy: 0.2822
Epoch 2/40
96/96 [==============================] - 4s 46ms/step - loss: 1.4404 - accuracy: 0.4396 - val_loss: 1.6730 - val_accuracy: 0.3574
Epoch 3/40
96/96 [==============================] - 4s 45ms/step - loss: 1.0311 - accuracy: 0.6171 - val_loss: 0.6290 - val_accuracy: 0.7684
Epoch 4/40
96/96 [==============================] - 4s 44ms/step - loss: 0.7934 - accuracy: 0.7267 - val_loss: 0.5612 - val_accuracy: 0.7730
Epoch 5/40
96/96 [==============================] - 4s 44ms/step - loss: 0.5406 - accuracy: 0.8084 - val_loss: 0.2994 - val_accuracy: 0.9018
Epoch 6/40
96/96 [==============================] - 4s 45ms/step - loss: 0.3689 - accuracy: 0.8727 - val_loss: 0.2670 - val_accuracy: 0.9064
Epoch 7/40
96/96 [==============================] - 4s 45ms/step - loss: 0.2896 - accuracy: 0.9098 - val_loss: 0.1748 - val_accuracy: 0.9448
Epoch 8/40
96/96 [======

In [7]:
model.evaluate(test)

21/21 [==============================] - 1s 19ms/step - loss: 0.0482 - accuracy: 0.9924


[0.04816286265850067, 0.9923781156539917]

In [ ]:
keras.models.save_model(model, "./models/vtid2_1.h5")